<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# LSEG Data Libraries for Python

**Financial Time Series Prediction &mdash; Recognizing Intraday Patterns**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical intraday data across via the Eikon Data API,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks` and
* how to apply machine learning (ML) techniques for time series prediction

## Importing Required Packages

In [3]:
import lseg.data as ld  # the LSEG Data Libraries for Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
from sklearn.svm import SVC  # sckikit-learn
import warnings; warnings.simplefilter('ignore')
from statsmodels.tsa.stattools import adfuller
import configparser as cp

The following **Python and package versions** are used.

In [4]:
import sys
print(sys.version)

3.9.6 (default, Aug 11 2023, 19:44:49) 
[Clang 15.0.0 (clang-1500.0.40.1)]


In [5]:
ld.__version__

'2.0.0'

In [6]:
np.__version__

'1.23.4'

In [7]:
pd.__version__

'2.0.2'

In [8]:
cf.__version__

'0.17.3'

## Connecting to LSEG Data Libraries

In [9]:
ld.open_session()

<lseg.data.session.Definition object at 0x151319b50 {name='workspace'}>

## Retrieving Intraday Data

We first define a **small universe of `RICS`** for which to retrieve data.

In [10]:
rics = [
    'SPY',  # S&P 500 ETF
    'AAPL.O',  # Apple stock
    'AMZN.O'  # Amazon stock
]

Second, **intraday data** is retrieved.

In [42]:
data = pd.DataFrame()
for ric in rics:
    data[ric] = ld.get_history(ric,  # the RICs
                     fields='TRDPRC_1',  # the required fields
                     start='2024-06-04 10:30:00',  # start date
                     end='2024-06-04 16:00:00', # end date
                     interval='minute')['TRDPRC_1']  # bar length 

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 329 entries, 2024-06-04 10:30:00 to 2024-06-04 15:59:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY     329 non-null    Float64
 1   AAPL.O  328 non-null    Float64
 2   AMZN.O  314 non-null    Float64
dtypes: Float64(3)
memory usage: 11.2 KB


In [44]:
data.head()  # first five rows

SPY  AAPL.O  AMZN.O
Timestamp                                  
2024-06-04 10:30:00  524.79  193.39  177.62
2024-06-04 10:31:00  524.86  193.29   177.6
2024-06-04 10:32:00   524.8  193.33   177.6
2024-06-04 10:33:00  524.93  193.38  177.62
2024-06-04 10:34:00   525.0  193.38    <NA>

In [45]:
data.tail()  # final five rows

SPY    AAPL.O    AMZN.O
Timestamp                                        
2024-06-04 15:55:00     525.6  193.8601  178.2601
2024-06-04 15:56:00    525.63   193.895    178.22
2024-06-04 15:57:00    525.48   193.864  178.2196
2024-06-04 15:58:00    525.41   193.895  178.2099
2024-06-04 15:59:00  525.2501  193.8499    178.12

In [46]:
data.dropna(inplace=True)

## Calculating the Log Returns

We next calculate the **log returns** in vectorized fashion.

In [47]:
rets = np.log(data / data.shift(1)).dropna()  # log returns in vectorized fashion

In [48]:
rets.head()

SPY    AAPL.O    AMZN.O
Timestamp                                        
2024-06-04 10:31:00  0.000133 -0.000517 -0.000113
2024-06-04 10:32:00 -0.000114  0.000207       0.0
2024-06-04 10:33:00  0.000248  0.000259  0.000113
2024-06-04 10:36:00  0.000324       0.0 -0.000113
2024-06-04 10:39:00 -0.000133       0.0 -0.000169

While **financial time series data** in general is not stationary ...

In [49]:
adfuller(data['AAPL.O'])  # test for stationarity of time series

(-3.281857770353911,
 0.015709167401074576,
 0,
 312,
 {'1%': -3.4514843502727306,
  '5%': -2.8708485956333556,
  '10%': -2.571729625657462},
 -496.4733938496015)

... the **log returns time series data** is in general.

In [50]:
adfuller(rets['AAPL.O'])  # test for stationarity of time series

(-9.355288687522243,
 8.090285187971355e-16,
 5,
 306,
 {'1%': -3.4519023023726696,
  '5%': -2.8710320399170537,
  '10%': -2.57182745012602},
 -3591.8000771237766)

## Plotting the Data

Using `Cufflinks`, we can plot the normalized financial time series as **line plots** for comparison.

In [51]:
data.normalize().iplot(kind='lines')

The frequeny distributions, i.e. the **histograms**, of the log returns per `RIC`.

In [52]:
rets.iplot(kind='histogram', subplots=True)

## Preparing Lagged Data

### Basic Idea

To create predictions for the financial time series analyzed, we work with **five lags**. The basic idea is that the historical (return) **values from the previous five days** are used to predict the value today. Consider the following simple data set.

In [53]:
n = 15
df = pd.DataFrame(np.arange(n), index=pd.date_range('2018-1-1', periods=n, freq='B'),
                 columns=['data'])
df

data
2018-01-01     0
2018-01-02     1
2018-01-03     2
2018-01-04     3
2018-01-05     4
2018-01-08     5
2018-01-09     6
2018-01-10     7
2018-01-11     8
2018-01-12     9
2018-01-15    10
2018-01-16    11
2018-01-17    12
2018-01-18    13
2018-01-19    14

The code below creates five additional columns with lagged data (one day back, two days back, ...).

In [54]:
lags = 5
for lag in range(1, lags + 1):
    df['lag_{}'.format(lag)] = df['data'].shift(lag)

In [55]:
df.dropna().astype(int)

data  lag_1  lag_2  lag_3  lag_4  lag_5
2018-01-08     5      4      3      2      1      0
2018-01-09     6      5      4      3      2      1
2018-01-10     7      6      5      4      3      2
2018-01-11     8      7      6      5      4      3
2018-01-12     9      8      7      6      5      4
2018-01-15    10      9      8      7      6      5
2018-01-16    11     10      9      8      7      6
2018-01-17    12     11     10      9      8      7
2018-01-18    13     12     11     10      9      8
2018-01-19    14     13     12     11     10      9

### Application

The code that follows derives the **lagged data** for every single `RIC`. First, a function that adds columns with lagged data to a `DataFrame` object.

In [56]:
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(rets[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        # creates the lagged data column with directional values
        df[col] = df[ric].shift(lag)
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

Second, the iterations over all `RICs`, using the `add_lags` function and storing the resulting `DataFrame` objects in a dictionary.

In [57]:
dfs = {}
for ric in rics:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [58]:
cols  # the column names for the lags

['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [59]:
dfs.keys()  # the keys of the dictonary

dict_keys(['SPY', 'AAPL.O', 'AMZN.O'])

In [60]:
dfs['AAPL.O'].head(7)

AAPL.O     lag_1     lag_2     lag_3     lag_4  \
Timestamp                                                               
2024-06-04 10:40:00       0.0       0.0       0.0  0.000259  0.000207   
2024-06-04 10:41:00 -0.000103       0.0       0.0       0.0  0.000259   
2024-06-04 10:42:00  0.000052 -0.000103       0.0       0.0       0.0   
2024-06-04 10:43:00 -0.000103  0.000052 -0.000103       0.0       0.0   
2024-06-04 10:44:00       0.0 -0.000103  0.000052 -0.000103       0.0   
2024-06-04 10:45:00 -0.000103       0.0 -0.000103  0.000052 -0.000103   
2024-06-04 10:46:00 -0.000052 -0.000103       0.0 -0.000103  0.000052   

                        lag_5  
Timestamp                      
2024-06-04 10:40:00 -0.000517  
2024-06-04 10:41:00  0.000207  
2024-06-04 10:42:00  0.000259  
2024-06-04 10:43:00       0.0  
2024-06-04 10:44:00       0.0  
2024-06-04 10:45:00       0.0  
2024-06-04 10:46:00 -0.000103

In [61]:
np.sign(dfs['AAPL.O'].head(7))

AAPL.O  lag_1  lag_2  lag_3  lag_4  lag_5
Timestamp                                                     
2024-06-04 10:40:00     0.0    0.0    0.0    1.0    1.0   -1.0
2024-06-04 10:41:00    -1.0    0.0    0.0    0.0    1.0    1.0
2024-06-04 10:42:00     1.0   -1.0    0.0    0.0    0.0    1.0
2024-06-04 10:43:00    -1.0    1.0   -1.0    0.0    0.0    0.0
2024-06-04 10:44:00     0.0   -1.0    1.0   -1.0    0.0    0.0
2024-06-04 10:45:00    -1.0    0.0   -1.0    1.0   -1.0    0.0
2024-06-04 10:46:00    -1.0   -1.0    0.0   -1.0    1.0   -1.0

In [62]:
2 ** lags  # number of patterns

32

## Implementing ML Algorithm

The matrix consisting of the lagged data columns is used to "predict" the next day's direction of movement of the `RIC` via the **support vector machine (SVM)** algorithm. This is a **classification algorithm** that is able to **learn from historical patterns** (5 lags) to predict whether an upwards movement is more likely or a downwards movement.

In [63]:
for ric in rics:
    model = SVC(C=100) # the ML model
    df = dfs[ric].copy()  # getting data for the RIC
    model.fit(np.sign(df[cols]), np.sign(df[ric]))  # model fitting
    dfs[ric]['position'] = model.predict(np.sign(df[cols]))  # prediction

The prediction value is either `+1` for an upwards movement or `-1` for a downwards movement. With regard to a using this as signals for a trading strategy, one **would go long for `+1` and go short for `-1`**.

In [64]:
for ric in rics:
    print('{:10} | {}'.format(ric, dfs[ric]['position'].values[:12]))

SPY        | [ 1.  1. -1.  1. -1.  1. -1. -1. -1.  1.  1.  1.]
AAPL.O     | [ 0. -1.  1. -1.  0. -1. -1.  1. -1.  1. -1.  1.]
AMZN.O     | [-1.  1. -1.  1.  1. -1. -1.  0.  1. -1.  0. -1.]


## Vectorized Backtesting

Let's backtest the performance of the ML-based trading strategies. Here, vectorization is used for convencience and speed. First, the **strategy returns** which result from multiplying the prediction or position values by the log returns of the respective `RIC`.

In [65]:
for ric in rics:
    dfs[ric]['strategy'] = dfs[ric]['position'] * dfs[ric][ric]

Second, the visualization of the **cumulative performance**.

In [66]:
for ric in rics:
    dfs[ric][[ric, 'strategy']].cumsum().apply(np.exp).iplot()

## Out-of-Sample Testing

Next, to get a more realistic picture of the real trading performance to be expected a **train test split** to implement **out-of-sample backtesting**.

In [67]:
split = int(len(data) / 2)

In [68]:
vspan = [{'x0': data.index[0], 'x1': data.index[split], 'color': 'green', 'fill': True, 'opacity': .2},
        {'x0': data.index[split], 'x1': data.index[-1], 'color': 'red', 'fill': True, 'opacity': .2}]

Roughly speaking, the **green part is taken for training**, the **red part for testing**.

In [69]:
data.normalize().iplot(vspan=vspan)

In [70]:
res = {}
for ric in rics:
    model = SVC(C=100) # the ML model
    df = dfs[ric].copy()  # getting data for the RIC
    split = int(len(df) / 2)
    train_x = np.sign(df[cols]).iloc[:split]
    train_y = np.sign(df[ric]).iloc[:split]
    test_x = np.sign(df[cols]).iloc[split:]
    test_y = df[ric].iloc[split:]
    model.fit(train_x, train_y)  # model fitting
    pred = model.predict(test_x)  # prediction
    strat = pred * test_y
    res[ric] = pd.DataFrame({ric: test_y,
                             'pred': pred,
                             'strategy': strat})

In [71]:
res['AAPL.O'].head()

AAPL.O  pred  strategy
Timestamp                                    
2024-06-04 13:26:00 -0.001082  -1.0  0.001082
2024-06-04 13:27:00  0.002421   1.0  0.002421
2024-06-04 13:28:00  0.001285   1.0  0.001285
2024-06-04 13:29:00 -0.000154  -1.0  0.000154
2024-06-04 13:30:00  0.001335   1.0  0.001335

In [72]:
for ric in rics:
    res[ric][[ric, 'strategy']].cumsum().apply(np.exp).iplot()

## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical intraday data (one minute bars)** via the Eikon Data API,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient,
* **machine learning (ML) techniques** such as SVM for classification are easily applied by the use of Python and
* that such techniques might be helpful in **predicting the direction of market movements** using a lag and pattern-based approach.

## LSEG Data Libraries Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.